<a href="https://colab.research.google.com/github/neal-logan/dsba6211-summer2024/blob/main/notebooks/dsba6211_summer2024_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with Prompt Engineering

This notebook contains examples and exercises to learning about prompt engineering. It was originally created by DAIR.AI | Elvis Saravia with modifications.

We will be using the [OpenAI APIs](https://platform.openai.com/) for all examples. I am using the default settings `temperature=0.7` and `top-p=1`

---

## 1. Prompt Engineering Basics: OpenAI API and configurations


Below we are loading the necessary libraries, utilities, and configurations.

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai==1.35.1
!pip install --upgrade langchain==0.2.5
!pip install --upgrade langchain-openai==0.1.8
!pip install langchain-community==0.2.5

In [2]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from google.colab import userdata

Load environment variables. Since I'm running this in a Colab notebook, I'm using `userdata.get()`. Just make sure to add your API Key into your Colab.

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*5wEevNCOf80GTHwptPTB4g.png)

Alternatively, you can use `python-dotenv` with a `.env` file with your `OPENAI_API_KEY` then load it.

In [3]:
# API configuration
openai.api_key = userdata.get("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

### Types of Endpoints

OpenAI (and other LLM) API's typically have two types API endpoints: completion and chat.

![](https://substackcdn.com/image/fetch/w_1456,c_limit,f_webp,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fccf3b37b-ad35-43b3-8cac-607a01473ba8_2719x508.png)

Source: [Generally Intelligent SubStack](https://generallyintelligent.substack.com/p/chat-vs-completion-endpoints)

Originally, the completions endpoint was the first endpoint. However, OpenAI has [announced](https://community.openai.com/t/completion-models-are-now-considered-legacy/656302) it is deprecating that endpoint.

`/completions` endpoint provides the completion for a single prompt and takes a single string as an input, whereas the `/chat/completions` provides the responses for a given dialog and requires the input in a specific format corresponding to the message history. Instead of taking a `prompt`, Chat models take a list of `messages` as input and return a model-generated message as output.



| MODEL FAMILIES               | EXAMPLES                                         | API ENDPOINT                               |
|------------------------------|--------------------------------------------------|--------------------------------------------|
| Newer models (2023–)         | gpt-4, gpt-4-turbo-preview, gpt-3.5-turbo        | https://api.openai.com/v1/chat/completions |
| Updated legacy models (2023) | gpt-3.5-turbo-instruct, babbage-002, davinci-002 | https://api.openai.com/v1/completions      |


Although the chat format is designed to make multi-turn conversations easy, it’s just as useful for single-turn tasks without any conversation.

We're going to the use the Chat `openai.chat.completions` endpoint.

For more details, check out [OpenAI's docs](https://platform.openai.com/docs/guides/text-generation/chat-completions-vs-completions).

In [4]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

response

ChatCompletion(id='chatcmpl-9iSBeMOt41muhV4ed7nzPelsoAlww', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Globe Life Field in Arlington, Texas.', role='assistant', function_call=None, tool_calls=None))], created=1720381810, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=53, total_tokens=70))

Typically, what we're most interested in is the `choices[0].message.content`:

In [5]:
response.choices[0].message.content

'The 2020 World Series was played at Globe Life Field in Arlington, Texas.'

To make things a bit easier to modify parameters, we can generalize the calls as a function, including parameters and messages.

In [6]:
def get_completion(params, messages):
    """ GET completion from openai api"""

    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

There's a variety of different parameters that are common with LLM's. Since LLM's are really just word predictors (auto-complete, distributions over vocabulary), they require [different sampling methods](https://huyenchip.com/2024/01/16/sampling.html) to get the next word.

| Parameter          | Description                                                                                                                                                   |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `model`            | Specifies the model to be used for generating responses. Different models may have different capabilities, size, and performance characteristics.             |

Here's [an outline](https://www.promptingguide.ai/introduction/settings) of different common LLM parameters.


In [7]:
!openai --help

usage: openai [-h] [-v] [-b API_BASE] [-k API_KEY] [-p PROXY [PROXY ...]] [-o ORGANIZATION]
              [-t {openai,azure}] [--api-version API_VERSION] [--azure-endpoint AZURE_ENDPOINT]
              [--azure-ad-token AZURE_AD_TOKEN] [-V]
              {api,tools,migrate,grit} ...

positional arguments:
  {api,tools,migrate,grit}
    api                 Direct API calls
    tools               Client side tools for convenience

options:
  -h, --help            show this help message and exit
  -v, --verbose         Set verbosity.
  -b API_BASE, --api-base API_BASE
                        What API base url to use.
  -k API_KEY, --api-key API_KEY
                        What API key to use.
  -p PROXY [PROXY ...], --proxy PROXY [PROXY ...]
                        What proxy to use.
  -o ORGANIZATION, --organization ORGANIZATION
                        Which organization to run as (will use your default organization if not
                        specified)
  -t {openai,azure}, --api-ty

In [8]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

In [9]:
# basic example

params = set_open_params(temperature = 0.7)

prompt = "Write a haiku about a beagle."

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [10]:
IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))

Curious nose sniffs,
Ears flopping in the sunlight,
Beagle's joyous dance.

### Temperature


| Parameter          | Description                                                                                                                                                   |
|--------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `temperature`      | Controls randomness in the response generation. A higher temperature results in more random responses, while a lower temperature produces more deterministic responses. |

Try to modify the temperature from 0 to 1.

> In terms of application, you might want to use a lower temperature value for tasks like fact-based QA to encourage more factual and concise responses. For poem generation or other creative tasks, it might be beneficial to increase the temperature value.

In [11]:
import re

def split_on_capital(text):
    return re.findall(r'[A-Z][^A-Z]*', text)

# Experiment with different temperature values
for temp in [0, 0.2, 0.4, 0.6, 0.8, 1]:
    params = set_open_params(temperature=temp)
    response = get_completion(params, messages)
    lines = split_on_capital(response.choices[0].message.content.strip())
    print(f"Response with temperature={temp}:\n")
    for line in lines:
        IPython.display.display(IPython.display.Markdown(line))
    print("-------------------\n")

Response with temperature=0:



Curious beagle nose


Sniffing out adventures near


Tail wags in delight

-------------------

Response with temperature=0.2:



...

-------------------

Response with temperature=1:



Beagle with long ears


Chasing scents through the forest


Playful and loyal

-------------------



[View how I used ChatGPT to iterate on this code](https://chat.openai.com/share/6211107a-d868-491b-8a74-ea4debb7a760)

> ### 🗒 Info: Is it possible to make LLM's deterministic and reproducibile?

> It's possible by setting a seed and setting temperature equal to 0. But as mentioned in [OpenAI's docs](https://cookbook.openai.com/examples/reproducible_outputs_with_the_seed_parameter), "it's important to note that while the seed ensures consistency, it does not guarantee the quality of the output."

### Top P

Used in nucleus sampling, it defines the probability mass to consider for token generation. A smaller `top_p` leads to more focused sampling.

* `top_p` computes the cumulative probability distribution, and cut off as soon as that distribution exceeds the value of `top_p`. **For example, a `top_p` of 0.3 means that only the tokens comprising the top 30% probability mass are considered.**

* `top_p` shrinks or grows the "pool" of available tokens to choose from, the domain to select over. 1=big pool, 0=small pool. Within that pool, each token has a probability of coming next.

Added in a `jinja2` template too (this is optional, but commonly used).

In [12]:
%%capture
!pip install textstat

In [13]:
import re
import json
from textstat import flesch_kincaid_grade
from jinja2 import Template

# Define the prompt using a Jinja template
template = Template("""
{
    "role": "user",
    "content": "{{ prompt }}"
}
""")
prompt = "Generate a unique and creative story idea involving time travel."

# Function to count unique words
def count_unique_words(text):
    words = re.findall(r'\b\w+\b', text.lower())
    return len(set(words))

# Generate the message using the template
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

for top_p in [0.1, 0.5, 0.95]:
    params = set_open_params(top_p=top_p)
    response = get_completion(params, messages)
    text = response.choices[0].message.content
    unique_word_count = count_unique_words(text)
    fk_score = flesch_kincaid_grade(text)
    print(f"Response with top_p={top_p}:\n")
    print(f"Unique word count: {unique_word_count}")
    print(f"Flesch-Kincaid Grade Level: {fk_score}")
    IPython.display.display(IPython.display.Markdown(text))
    print("-------------------\n")


Response with top_p=0.1:

Unique word count: 129
Flesch-Kincaid Grade Level: 10.3


In the year 3021, time travel has become a common form of entertainment for the wealthy elite. The Time Travel Corporation offers exclusive trips to different eras in history, allowing clients to witness major events firsthand.

One day, a young woman named Eliza wins a contest to travel back to the year 1920 and attend a lavish party hosted by the infamous gangster Al Capone. Excited for the adventure, Eliza steps into the time machine and is transported back in time.

However, when she arrives in 1920, Eliza quickly realizes that something has gone wrong. Instead of being a mere observer, she finds herself trapped in the past, unable to return to her own time. Desperate to find a way back home, Eliza must navigate the dangerous world of Prohibition-era Chicago while trying to avoid altering the course of history.

As she struggles to survive in this unfamiliar time period, Eliza discovers that her presence in the past may have unintended consequences. She must race against the clock to find a way back to the future before her actions change the course of history forever.

Along the way, Eliza forms an unlikely alliance with a charming bootlegger named Jack, who helps her navigate the treacherous underworld of 1920s Chicago

-------------------

Response with top_p=0.5:

Unique word count: 129
Flesch-Kincaid Grade Level: 8.5


In the year 3025, time travel has become a common form of entertainment for the wealthy elite. The Time Travel Corporation offers exclusive trips to different time periods for those who can afford it. 

One day, a young woman named Amelia wins a contest to travel back in time to the year 1920. Excited for the opportunity to experience the Roaring Twenties, Amelia eagerly steps into the time machine. However, when she arrives in 1920, she quickly realizes that something is not right. 

Instead of the glamorous parties and flapper dresses she was expecting, Amelia finds herself in a war-torn world. The year 1920 is not the carefree era she had imagined, but rather a time of great hardship and suffering. 

Determined to find out what went wrong, Amelia sets out to uncover the truth behind this alternate version of history. Along the way, she meets a group of rebels who are fighting against the oppressive government that has taken control of the timeline. 

As Amelia delves deeper into the mystery, she discovers that the Time Travel Corporation has been manipulating the past for their own gain, altering events to suit their own agenda. With the help of the rebels, Amelia must find a way to restore the timeline to its original state

-------------------

Response with top_p=0.95:

Unique word count: 131
Flesch-Kincaid Grade Level: 11.5


In the year 2150, time travel has become a common technology used for historical research. A team of archaeologists and historians are tasked with traveling back in time to ancient Egypt to uncover the mysteries of the pyramids. However, during their expedition, they accidentally disrupt a crucial event in history, causing a ripple effect that alters the course of civilization.

As they struggle to repair the damage they've done, they realize that their actions have unleashed a dangerous force that threatens to unravel the fabric of time itself. The team must race against the clock to fix their mistake and restore the timeline before it's too late.

But as they delve deeper into the secrets of the past, they uncover a shocking truth about the origins of time travel and the dark forces that seek to control it. With the fate of the world hanging in the balance, the team must make a desperate decision that will test their loyalty, bravery, and belief in the power of humanity.

Ultimately, they must confront their own inner demons and fears as they navigate a treacherous journey through time and space, facing enemies both ancient and modern. Will they be able to set things right and save the future, or will they be doomed to repeat the mistakes of the past?

-------------------



> **If you are looking for exact and factual answers keep this low. If you are looking for more diverse responses, increase to a higher value.** If you use Top P it means that only the tokens comprising the top_p probability mass are considered for responses, so a low top_p value selects the most confident responses. This means that a high top_p value will enable the model to look at more possible words, including less likely ones, leading to more diverse outputs. **The general recommendation is to alter temperature or Top P but not both.**

In [16]:
prompt = "What day is it?"

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

for top_p in [0.5, 0.8, 0.95]:
    params = set_open_params(top_p=top_p)
    response = get_completion(params, messages)
    print(f"Response with top_p={top_p}:\n")
    IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))
    print("-------------------\n")

Response with top_p=0.5:



Today is Thursday.

-------------------

Response with top_p=0.8:



Today is Monday.

-------------------

Response with top_p=0.95:



I am an AI digital assistant and do not have the ability to know the current date. Please check your device or calendar for the current day.

-------------------



### Max Tokens

Defines the maximum length of the generated response measured in tokens (words or pieces of words). It helps in controlling the verbosity of the response.

* **Prompt**: "Explain how photosynthesis works in plants."
* **Task**: Run the API three times with `max_tokens` set to 50, 100, and 200, respectively.

In [17]:
# Experiment with different max_tokens values
prompt = "Explain how photosynthesis works in plants."

messages = [
    {
        "role": "user",
        "content": prompt
    }
]


for max_tokens in [50, 100, 200]:
    params = set_open_params(max_tokens=max_tokens)
    response = get_completion(params, messages)
    print(f"Response with max_tokens={max_tokens}:\n")
    IPython.display.display(IPython.display.Markdown(response.choices[0].message.content))
    print("-------------------\n")

Response with max_tokens=50:



Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose. This process involves several key steps:

1. Absorption of light: Chlorophyll, a green pigment

-------------------

Response with max_tokens=100:



Photosynthesis is the process by which plants, algae, and some bacteria convert sunlight, carbon dioxide, and water into glucose (sugar) and oxygen. This process takes place in the chloroplasts of plant cells.

During photosynthesis, chlorophyll (a green pigment in plants) absorbs sunlight and uses it to convert carbon dioxide (CO2) from the air and water (H2O) from the soil into glucose (C6H12O6) and oxygen (O2).

-------------------

Response with max_tokens=200:



Photosynthesis is the process by which plants, algae, and some bacteria convert sunlight, carbon dioxide, and water into glucose (sugar) and oxygen. This process occurs in the chloroplasts of plant cells.

During photosynthesis, chlorophyll (a pigment found in chloroplasts) absorbs sunlight and uses it to convert carbon dioxide and water into glucose. This process involves two main stages: the light-dependent reactions and the light-independent reactions (also known as the Calvin cycle).

In the light-dependent reactions, sunlight is absorbed by chlorophyll and used to split water molecules into oxygen and hydrogen ions. The oxygen is released into the atmosphere as a byproduct, while the hydrogen ions are used to generate chemical energy in the form of ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleotide phosphate).

In the light-independent reactions (Calvin cycle), ATP and NADPH are used to convert carbon dioxide into glucose. This process involves

-------------------



### 1.1 Text Summarization

In [18]:
params = set_open_params(temperature=0.7)

#Original prompt
# prompt = """Explain the below in one sentence: Antibiotics are a type of medication \
#           used to treat bacterial infections. They work by either killing the bacteria \
#           or preventing them from reproducing, allowing the body's immune system to \
#           fight off the infection. Antibiotics are usually taken orally in the form \
#           of pills, capsules, or liquid solutions, or sometimes administered intravenously. \
#           They are not effective against viral infections, and using them inappropriately \
#           can lead to antibiotic resistance."""


#This modified prompt worked pretty well
# prompt = """Explain the below in one sentence: Antibiotics are a type of medication \
#           used to treat bacterial infections. They work by either killing the bacteria \
#           or preventing them from reproducing, allowing the body's immune system to \
#           fight off the infection. Antibiotics are usually taken orally in the form \
#           of pills, capsules, or liquid solutions, or sometimes administered intravenously. \
#           They are not effective against viral infections, and using them inappropriately \
#           can lead to antibiotic resistance. ELI5 literally"""

#Modified prompt
prompt = """Explain the below in one sentence: Antibiotics are a type of medication \
          used to treat bacterial infections. They work by either killing the bacteria \
          or preventing them from reproducing, allowing the body's immune system to \
          fight off the infection. Antibiotics are usually taken orally in the form \
          of pills, capsules, or liquid solutions, or sometimes administered intravenously. \
          They are not effective against viral infections, and using them inappropriately \
          can lead to antibiotic resistance. Explain like I am 5"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Antibiotics are special medicines that help our bodies fight off bad bacteria when we get sick, but they don't work against viruses and we have to be careful to only use them when the doctor says it's okay.

> **Exercise**: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

### 1.2 Question Answering

In [45]:
# prompt = """Answer the question based on the context below. \
#  Keep the answer short and concise. \
#  Respond 'Unsure about answer' if not sure about the answer. \
#  Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. \
#  There, scientists generated an early version of the antibody, dubbed OKT3. \
#  Originally sourced from mice, the molecule was able to bind to the surface of T cells \
#  and limit their cell-killing potential. In 1986, it was approved to help prevent organ \
#  rejection after kidney transplants, making it the first therapeutic antibody allowed \
#  for human use. \
#  Question: What was OKT3 originally sourced from? \
#  Answer:"""

#This prompt often still answers "mice" or "mice (Unsure about answer)" but
#also seems to produce the desired result 1/3 of the time (based on ~10 queries)
# prompt = """Answer the question based on the context below. \
#  Keep the answer short and concise. \
#  Respond 'Unsure about answer' if the answer isn't absolutely, completely certain. \
#  Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. \
#  There, scientists generated an early version of the antibody, dubbed OKT3. \
#  Supposedly originally sourced from mice, the molecule was able to bind to the surface of T cells \
#  and limit their cell-killing potential. However the original source can't be verified. \
#  In 1986, it was approved to help prevent organ \
#  rejection after kidney transplants, making it the first therapeutic antibody allowed \
#  for human use. \
#  Question: What was OKT3 originally sourced from? \
#  Answer:"""


#Adding the phrase "OK3 is not currently obtained from mice." led the model to
#admit it's unsure about the origin of OKT3 consistently, despite having no direct
#bearing on that question
prompt = """Answer the question based on the context below. \
 Keep the answer short and concise. \
 Respond 'Unsure about answer' if the answer isn't absolutely, completely certain. \
 Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. \
 There, scientists generated an early version of the antibody, dubbed OKT3. \
 Supposedly originally sourced from mice, the molecule was able to bind to the surface of T cells \
 and limit their cell-killing potential. However the original source can't be verified. \
 In 1986, it was approved to help prevent organ \
 rejection after kidney transplants, making it the first therapeutic antibody allowed \
 for human use. OKT3 is not currently obtained from mice. \
 Question: What was OKT3 originally sourced from? \
 Answer:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


Unsure about answer

Context obtained from here: https://www.nature.com/articles/d41586-023-00400-x

> **Exercise**: Edit prompt and get the model to respond that it isn't sure about the answer.

### 1.3 Text Classification

In [50]:
# Original prompt
# prompt = """Classify the text into neutral, negative or positive. \
#                                                   \
#             Text: I think the food was amazing!   \
#                                                   \
#             Sentiment:"""

prompt = """Classify the text into neutral, negative or positive. \
            Then explain the reasons for the classification.      \
                                                  \
            Text: I think the food was amazing!   \
                                                  \
            Sentiment:                            \
            Explanation:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Positive

Explanation: The use of the word "amazing" indicates a positive sentiment towards the food. The overall tone of the text is enthusiastic and satisfied.

> **Exercise**: Modify the prompt to instruct the model to provide an explanation to the answer selected.

### 1.4 Role Playing

In [59]:
#Original prompt
# prompt = """The following is a conversation with an AI research assistant. \
# The assistant tone is technical and scientific. \
#                                                 \
# Human: Hello, who are you? \
# AI: Greeting! I am an AI research assistant. How can I help you today? \
# Human: Can you tell me about the creation of blackholes? \
# AI:"""

# Direction to be concise - interestingly, this was achieved despite not
# making the response length limit explicit
prompt = """The following is a conversation with an AI research assistant. \
The assistant tone is technical and scientific. The assistant limits the length\
of its responses to the absolute minimum number of words necessary to answer   \
the question.                                                                  \
Human: Hello, who are you? \
AI: Greeting! I am an AI research assistant. How can I help you today? \
Human: Can you tell me about the creation of blackholes? \
AI:"""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Black holes form from collapsing stars.

> **Exercise**: Modify the prompt to instruct the model to keep AI responses concise and short.

### 1.5 Code Generation

In [60]:
#Original prompt
# prompt = """Develop a small Python API that functions as a virtual plant care assistant."""

prompt = """Develop a small Python API that functions as a virtual plant care assistant."""


# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Here is a simple Python API that acts as a virtual plant care assistant:

```python
class PlantCareAssistant:
    def __init__(self, plant_name):
        self.plant_name = plant_name
        self.water_level = 100
        self.sunlight_level = 100

    def water_plant(self, amount):
        self.water_level += amount
        if self.water_level > 100:
            self.water_level = 100
        return f"Watered {self.plant_name}. Water level is now {self.water_level}%."

    def provide_sunlight(self, amount):
        self.sunlight_level += amount
        if self.sunlight_level > 100:
            self.sunlight_level = 100
        return f"Provided sunlight to {self.plant_name}. Sunlight level is now {self.sunlight_level}%."

    def check_plant_status(self):
        return f"{self.plant_name} status - Water level: {self.water_level}%, Sunlight level: {self.sunlight_level}%."

# Example usage
plant_assistant = PlantCareAssistant("Rose")
print(plant_assistant.water_plant(20))
print(plant_assistant.provide_sunlight(30))
print(plant_ass

### 1.6 Reasoning

In [79]:
# Original prompt
# prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. \
#                                                                                               \
# Solve by breaking the problem into steps.                                                     \
#                                                                                               \
# First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

#Modified prompt
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. \
Solve by breaking the problem into steps. At each step, explain what you are doing and why. \
First, identify the odd numbers. Second, add the odd numbers together. \
Finally, indicate whether the result is odd or even. Put each step on a new line. """


# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

- Identify the odd numbers in the group: 15, 5, 13, 7, 1.
- Add the odd numbers together: 15 + 5 + 13 + 7 + 1 = 41.
- The result, 41, is an odd number.

> **Exercise**: Improve the prompt to have a better structure and output format.

### 1.7 Brain storming

In [ ]:
prompt = """Generate 10 different names for a grunge rock bands based on dog names. Keep it to only 2 word band names."""

# Generate the message using the template and convert it to a dictionary
messages_str = template.render(prompt=prompt)
messages = [json.loads(messages_str)]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

1. Pug Riot
2. Husky Haze
3. Bulldog Rebellion
4. Rottweiler Rage
5. Boxer Chaos
6. Doberman Distortion
7. Beagle Fury
8. Pitbull Pandemonium
9. Dalmatian Anarchy
10. Great Dane Grit

### 1.8 JSON generation

Another new feature in OpenAI is [JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode).



In [ ]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Generate 5 doctor's notes as a 'text' field that precribes drugs, dosages, form, duration, and frequency. Then in JSON have an 'entities' nested dictionary with each of the entities"}
  ]
)
print(response.choices[0].message.content)

{
    "doctor_notes": [
        {
            "text": "Prescription: Advil 200mg tablets, take 1 tablet by mouth every 4-6 hours as needed for pain. Duration: 7 days.",
            "entities": {
                "drug": "Advil",
                "dosage": "200mg",
                "form": "tablets",
                "route": "by mouth",
                "frequency": "every 4-6 hours",
                "duration": "7 days"
            }
        },
        {
            "text": "Prescription: Amoxicillin 500mg capsules, take 1 capsule by mouth twice daily with food. Duration: 10 days.",
            "entities": {
                "drug": "Amoxicillin",
                "dosage": "500mg",
                "form": "capsules",
                "route": "by mouth",
                "frequency": "twice daily",
                "duration": "10 days"
            }
        },
        {
            "text": "Prescription: Zyrtec 10mg tablets, take 1 tablet by mouth daily at bedtime. Duration: 14 days.",
      

> ### 🗒 Info
> - When using JSON mode, always instruct the model to produce JSON via some message in the conversation, for example via your system message. If you don't include an explicit instruction to generate JSON, the model may generate an unending stream of whitespace and the request may run continually until it reaches the token limit. To help ensure you don't forget, the API will throw an error if the string "JSON" does not appear somewhere in the context.

> - The JSON in the message the model returns may be partial (i.e. cut off) if finish_reason is length, which indicates the generation exceeded max_tokens or the conversation exceeded the token limit. To guard against this, check finish_reason before parsing the response.

> - JSON mode will not guarantee the output matches any specific schema, only that it is valid and parses without errors.

## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

[Default v2 Prompt from Prodigy](https://prodi.gy/docs/large-language-models#more-config).

In [ ]:
prompt = """
You are an expert Named Entity Recognition (NER) system. Your task is to accept Text as input and extract named entities for the set of predefined entity labels.

From the Text input provided, extract named entities for each label in the following format:

DISH: <comma delimited list of strings>
INGREDIENT: <comma delimited list of strings>
EQUIPMENT: <comma delimited list of strings>

Below are definitions of each label to help aid you in what kinds of named entities to extract for each label.
Assume these definitions are written by an expert and follow them closely.

DISH: Extract the name of a known dish.
INGREDIENT: Extract the name of a cooking ingredient, including herbs and spices.
EQUIPMENT: Extract any mention of cooking equipment. e.g. oven, cooking pot, grill

Below are some examples (only use these as a guide):

Text:
'''
You can't get a great chocolate flavor with carob.
'''

INGREDIENT: carob

Text:
'''
You can probably sand-blast it if it's an anodized aluminum pan.
'''

INGREDIENT:
EQUIPMENT: anodized aluminum pan


Here is the text that needs labeling:

Text:
'''
In Silicon Valley, a Voice of Caution Guides a High-Flying Uber
'''
"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

DISH: 
INGREDIENT: 
EQUIPMENT: Silicon Valley, Uber

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The odd numbers in the group are 15, 5, 13, 7, and 1. Their sum is 15 + 5 + 13 + 7 + 1 = 41, which is an odd number.

In [ ]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The answer is True.

### 2.3 Chain-of-Thought (CoT) Prompting

In [ ]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)



Initially, you bought 10 apples. 
After giving 2 to the neighbor, you had 10 - 2 = 8 apples remaining. 
After giving 2 to the repairman, you had 8 - 2 = 6 apples remaining. 
Then you bought 5 more apples, so you had 6 + 5 = 11 apples. 
After eating 1 apple, you remained with 11 - 1 = 10 apples.

So, you remained with 10 apples in total.

### 2.5 Self-Consistency
As an **optional** exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency) and try them here.

### 2.6 Generate Knowledge Prompting

As an **optional** exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting) and try them here.

### 3 LangChain

This is adopted from [this notebook](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-chatgpt-langchain.ipynb).

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
# chat mode instance
chat = ChatOpenAI(temperature=0)

In [ ]:
USER_INPUT = "I love programming."
FINAL_PROMPT = """Classify the text into neutral, negative or positive.

Text: {user_input}.
Sentiment:"""

chat.invoke([HumanMessage(content=FINAL_PROMPT.format(user_input=USER_INPUT))])

AIMessage(content='Positive', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 27, 'total_tokens': 28}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7a88107b-5e0b-46f4-ad32-6b77dac00ff5-0', usage_metadata={'input_tokens': 27, 'output_tokens': 1, 'total_tokens': 28})

---